In [3]:
import pandas as pd
import numpy as np
import pdb

In [4]:
fac_trace = pd.read_csv('/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/data/2019-12-18_facility_trace.csv', index_col=0)
fac_trace.columns = np.arange(367)
floor_trace = pd.read_csv('/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/data/2019-12-18_floor_trace.csv', index_col=0)
floor_trace.columns = np.arange(367)
room_trace = pd.read_csv('/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/data/2019-12-18_room_trace.csv', index_col=0)
room_trace.columns = np.arange(367)

In [5]:
intake_data = pd.read_csv("/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/network/intake_data.csv", index_col=0)
# tests for invase infections, may not necessarily be positive
tests = pd.read_csv("/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/network/infections.csv", index_col=0)

In [6]:
# handle multiple tests same-day per patient
# unresolved issue: what if someone gets a negative test after a positive test?
tests = tests.reset_index().sort_values(["index", "test_time", "Carbapenem_R"])\
    .drop_duplicates(["index", "test_time"], keep="last").set_index("index")

In [7]:
# generate dataframe of known statuses
n = fac_trace.shape[0]
T = fac_trace.shape[1]
# this might be more like...known events?
known_status = pd.DataFrame(index=intake_data.index, columns=range(T))
for t in range(T):
    # statuses upon intake
    for pid, result in intake_data[intake_data["date"] == t]["crkp"].items():
        known_status.loc[pid, t] = result
    # statuses upon test for infection
    for pid, result in tests[tests["test_time"] == t]["Carbapenem_R"].items():
        known_status.loc[pid, t] = result
    # what about people who are already in facility?
    # does not account for this ... 
    # can handle this downstream

In [8]:
patient_weeks = fac_trace[fac_trace > 0].notna().sum().sum()
n_known_statuses = known_status.notna().sum().sum()
n_known_statuses / patient_weeks

0.03132326126595467

In [9]:
# output
# how to grow this out..column by column?
transmission_status = pd.DataFrame(index=known_status.index, columns=range(T))
# do i want to fully account for the first week?
transmission_status[0] = known_status[0]

In [10]:
# dummy

status = transmission_status[0]
facility = fac_trace[0]
floor = floor_trace[0]
room = room_trace[0]
admitted = set(facility[facility!= 0].index)
susceptible = set(status[status==0].index) & admitted
infected = set(status[status==1].index) & admitted

In [11]:
# prior simulation
# beta is 8 dimensional: baseline rate, six floors, room rate
beta = np.exp(np.random.normal(scale=3, size=8))

In [30]:
class CRKPTransmissionSimulator:
    def __init__(self, facility_trace, floor_trace, room_trace, known_status, beta):
        self.fac_tr = facility_trace
        self.floor_tr = floor_trace
        self.room_tr = room_trace
        self.known = known_status
        self.beta = beta
        self.T = self.fac_tr.shape[1]

    def simulate_data(self, N):
        simulations = []
        for _ in range(N):
            # can try to calculate summary statistics, here
            simulations.append(self.simulate())

    def _simulate(self):
        transmission_status = pd.DataFrame(index=known_status.index, columns=range(T))
        # TODO: do i want to hard code week 0?
        transmission_status[0] = known_status[0]
        y = 1/0
        for t in range(1, T):
            status = transmission_status[t-1]
            facility = self.fac_tr[t-1]
            admitted = set(facility[facility!= 0])
            floor = self.floor_tr[t-1]
            room = self.room_tr[t-1]
            # requirement: you were susceptible in the previous timestep 
            # and are not discharged this week
            susceptible = set(status[status==0].index) & admitted
            for j in susceptible:
                hazard_j = 0
                # should not be facility..
                hazard_j += status[status == 1].count() * self.beta[0]
                floor_num = floor.loc[j]
                hazard_j += status[(status == 1) & (floor == floor_num)].count() \
                      * self.beta[floor_num]
                room_num = room.loc[j]
                hazard_j += status[(status == 1) & (room == room_num)].count() \
                    * self.beta[-1]

                p_j = 1 - np.exp(hazard_j)

                Y_j = np.random.binomial(1, p_j)
                transmission_status.loc[j, t] = Y_j


            # overwrite with known statuses
            #
            

                # TODO: overwrite this with...known statuses?

In [28]:
simulator = CRKPTransmissionSimulator(
    fac_trace, floor_trace, room_trace,
    known_status, beta
)

In [34]:
hex = "booyeah"

1/0

ZeroDivisionError: division by zero

In [31]:
simulator._simulate()

ZeroDivisionError: division by zero